# Analyzing risk of defaulting on bank loans

This project is meant to help a bank's loan division on evaluating a borrower's score using statistics of customers. The data of more than 20,000 clients was used as reference and they were analyzed with regards to age, monthly income, marital status, income type and purpose for asking a loan. The customer's score may be used as a trustworthiness index to repay a loan. It may help to define the client's eligibility for a loan, total credit available or interest rates for their loans.

The present report contains the following sections:

- Checking data, to learn about it and find inconsistent information
- Preprocessing, to make up for errors found
- Dictionaries, to standardize IDs and create reference tables
- Categorization, to better visualize the data
- Re-checking data, after fixing issues to verify it is correct
- Analysis, making using of pivot tables to answer questions
- Conclusion, summing up the results gathered



## Checking the data

### Introduction

The first step is to look into the data to better understand what information is available to gather results and reach our problem question: **is there any relation between a customer's profile characteristics and their risk of defaulting on a loan?**

The data available is very likely to be missing information; to have artifacts or duplicates; or to have inconsistent categorical variables. After reading through it, we will list what must be done to fix these issues and prepare for the next section.

Below is a reference for what details are available for each:

- children: the number of children in the family
- days_employed: how long the customer has been working
- dob_years: the customer’s age
- education: the customer’s education level
- education_id: identifier for the customer’s education
- family_status: the customer’s marital status
- family_status_id: identifier for the customer’s marital status
- gender: the customer’s gender
- income_type: the customer’s income type
- debt: whether the customer has ever defaulted on a loan
- total_income: monthly income
- purpose: reason for taking out a loan

### Data info

In [1]:
import pandas as pd 

try:
    df = pd.read_csv('./datasets/credit_scoring_eng.csv')
except:
    df = pd.read_csv('credit_scoring_eng.csv')
    
df.info()
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


Let's check the percentage of missing values for each column:

In [2]:
df.isnull().sum()/df.count()

children            0.000000
days_employed       0.112346
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.112346
purpose             0.000000
dtype: float64

In [3]:
print(f"The unique values for number of children are {df['children'].unique()}")
print()
print('Here is a short description of the days employed column:')
print(f"{df['days_employed'].describe()}")
print()
print(f"And here is a description of the age of customers \n {df['dob_years'].describe()}")
print()
print(f"Unique values in 'education': \n {df['education'].unique()}")
print()
print(f"Unique values in 'family_status': {df['family_status'].unique()}")
print()
print(f"Unique values in 'income_type': {df['income_type'].unique()}")
print()
print(f"Unique values in 'debt': {df['debt'].unique()}")
print()
print(f"Number of duplicates: {df.duplicated().sum()}")
print(f"A few duplicated rows: \n {df[df.duplicated()].head(3)}")
print()
print(f"Artifacts in gender: {df[~df['gender'].isin(['F', 'M', 'X'])]['gender'].unique()}")
print()
print(f"Artifacts in 'total_income': {df[df['total_income'] <= 0]['total_income'].value_counts().head()}")

The unique values for number of children are [ 1  0  3  2 -1  4 20  5]

Here is a short description of the days employed column:
count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64

And here is a description of the age of customers 
 count    21525.000000
mean        43.293380
std         12.574584
min          0.000000
25%         33.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

Unique values in 'education': 
 ["bachelor's degree" 'secondary education' 'Secondary Education'
 'SECONDARY EDUCATION' "BACHELOR'S DEGREE" 'some college'
 'primary education' "Bachelor's Degree" 'SOME COLLEGE' 'Some College'
 'PRIMARY EDUCATION' 'Primary Education' 'Graduate Degree'
 'GRADUATE DEGREE' 'graduate degree']

Unique values in 'family_status': ['married' 'civil partnership' 

### Conclusion

After using the <code>.describe()</code> and <code>.unique()</code> methods for each column of the data, we could find out problems with the data. Fixing these will be our first step before we can analyze the information and get to the end results.

We have a list of things to be dealt with:
- There are missing values in 'days_employed' and 'total_income'
- 'days_employed' and 'total_income' could be int type
- 'children' has artifacts
- 'days_employed' has negative values and positive data has artifacts (values are too high)
- 'dob_years' has age 0 entries
- 'education' entries need to be standardized
- 'education_id' should use a dictionary
- 'family_id' should use a dictionary
- 'income_type' could use a dictionary and has artifacts ('entrepreneur')
- 'purpose' needs stemming
- a few rows with missing values are duplicated


## Data preprocessing

### Introduction

In this section, we will start to fix the issues pointed out in section 1 to prepare data for analysis. We will start with missing values. Here are some options available to deal with the missing data:
 1. If the missing data comprises a small percentage (5~10%) of the data, deleting it can be an option. 
 2. If it is possible to find a reason why this data came in a wrong format, it might be restored to its correct value
 3. If the data is not the main variable of interest, the missing values can be filled and still be useful for the anaylis.

### Processing missing values

Missing values were found in 'days_employed' and 'total_income'. These are numerical values, they represent a quantitative data. This type of data can generally be replaced by their mean or median if they are not the main variable of analysis. We can identify the percentage of missing data, how this missing data affects the main variable of interest and then correctly fill it.

In [4]:
df[df['days_employed'].isnull()].isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [5]:
df[df['income_type'].isin(['unemployed', 'student'])]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
3133,1,337524.466835,31,secondary education,1,married,0,M,unemployed,1,9593.119,buying property for renting out
9410,0,-578.751554,22,bachelor's degree,0,unmarried,4,M,student,0,15712.260,construction of own property
14798,0,395302.838654,45,Bachelor's Degree,0,civil partnership,1,F,unemployed,0,32435.602,housing renovation


In [6]:
df.loc[df['income_type'] == 'unemployed', 'days_employed'] = 0

In [7]:
miss = df[df['days_employed'].isnull()]
print(f"The percentage of customers with debts in missing data is {miss[miss['debt'] == 1]['debt'].count()/miss.shape[0]:.2%}")
print(f"The percentage of customers with debts in whole data is {df[df['debt'] == 1]['debt'].count()/df.shape[0]:.2%}")

The percentage of customers with debts in missing data is 7.82%
The percentage of customers with debts in whole data is 8.09%


To fill missing values with the mean or media, the data for days employed must first be fixed. It has data as negative numbers and absurdly high numbers, which will affect these values for the column.

In [8]:
print(f"Percentage of negative days employed is {df[df['days_employed'] < 0]['days_employed'].count()/df.shape[0]:.2%}")

Percentage of negative days employed is 73.90%


In [9]:
positive_days = df[df['days_employed'] > 0]
print(positive_days[positive_days['days_employed'] < 300000]['days_employed'])
print(f"Percentage of artifacts as positive days is {df[df['days_employed'] > 300000]['days_employed'].count()/df.shape[0]:.2%}")

Series([], Name: days_employed, dtype: float64)
Percentage of artifacts as positive days is 16.00%


In [10]:
df.loc[df['days_employed'] > 0, 'days_employed'] = None
df['days_employed'] = df['days_employed'].abs()

In [11]:

days_employed_mean = df['days_employed'].mean()
days_employed_median = df['days_employed'].median()
income_mean = df['total_income'].mean()
income_median = df['total_income'].median()

print(f'The mean of days employed is {days_employed_mean:.1f}')
print(f'The median of days employed is {days_employed_median:.1f}')
print(f'The mean of total income is {income_mean:.1f}')
print(f'The median of total income is {income_median:.1f}')

The mean of days employed is 2352.7
The median of days employed is 1629.9
The mean of total income is 26787.6
The median of total income is 23202.9


In [12]:
df['days_employed'] = df['days_employed']
df['total_income'] = df['total_income']
df['days_employed'].fillna(days_employed_median, inplace=True)
df['total_income'].fillna(income_mean, inplace=True)

print(f"New mean of days employed is {df['days_employed'].mean():.1f}")

New mean of days employed is 2164.1


### Conclusion of missing values

- Some rows were missing days employed and monthly income altogether
- 10% of the data has missing values for these columns
- 73% of the data had negative values for 'days_employed', so they must be the negative of real data
- 16% of the data had absurdly high values for 'days_employed', it doesn't seem random but I couldn't identify what caused it
- The percentage of customers who have defaulted is the same in missing data and globally
- Missing data will be filled with median and mean values, depending on outliers
- Artifacts in 'days_employed' have been taken care of before filling
    - For 'income_type' unemployed, the values were turned to zero
- 'days_employed' has significant outliers, so missing values will be filled with median: 1630
- 'total_income' doesn't have significant outliers, so missing values will be filled with mean: 26788


###  Data type replacement

In this section, data types 'float64' will be changed to 'int64' for better visualization. The data used for customer analysis doesn't need to get as detailed as decimal places. Artifacts, which are incorrect values mixed with correct data, will also get analyzed and correctly replaced or deleted, according to the reason why it was created and percentage of ocurrence in the data.

To turn float numbers into int, they will be first rounded and then converted using the <code>.round().astype()</code> methods.

In [13]:
df['days_employed'] = df['days_employed'].round().astype('int64')
print(f"'days_employed' dtype is {df['days_employed'].dtype}")

df['total_income'] = df['total_income'].round().astype('int64')
df['total_income'].dtype
print(f"'total_income' dtype is {df['total_income'].dtype}")


'days_employed' dtype is int64
'total_income' dtype is int64


### Correction of artifacts

Artifacts must be analyzed individually and then replaced by suitable values. Examples are: Age = 0 for a customer, or a negative number of children. They were identified in section 1, and we will correct them here.

In [14]:
df.loc[df['children'] == 20, 'children'] = 2
df.loc[df['children'] == -1, 'children'] = 1
df['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [15]:
print(df['dob_years'].value_counts().sort_index().head())
age_mean = df[df['dob_years'] != 0]['dob_years'].mean()
age_median = df[df['dob_years'] != 0]['dob_years'].median()

print(f"The mean of age is {age_mean:.1f}")
print(f"The median of age is {age_median}")
df.loc[df['dob_years'] == 0, 'dob_years'] = int(age_mean)
df['dob_years'].value_counts().sort_index().head()

0     101
19     14
20     51
21    111
22    183
Name: dob_years, dtype: int64
The mean of age is 43.5
The median of age is 43.0


19     14
20     51
21    111
22    183
23    254
Name: dob_years, dtype: int64

In [16]:
df['education'] = df['education'].str.lower()
df['education'].value_counts()

secondary education    15233
bachelor's degree       5260
some college             744
primary education        282
graduate degree            6
Name: education, dtype: int64

In [17]:
df.loc[df['income_type'] == 'entrepreneur', 'income_type'] = 'business'
df['income_type'].value_counts()

employee                       11119
business                        5087
retiree                         3856
civil servant                   1459
unemployed                         2
paternity / maternity leave        1
student                            1
Name: income_type, dtype: int64

In [18]:
df.loc[df['gender'] == 'XNA', 'gender'] = 'X'
df['gender'].unique()

array(['F', 'M', 'X'], dtype=object)

### Conclusion
- a few entries had values in 'children' as 20 or -1, they were replaced by 2 and 1 respectively
- 'days_employed' dtypes were cast to int and negative values were replaced by their absolute
- 'total_income' dtypes were cast to int
- 'dob_years' entries with age 0 were replaced by mean, because mean and median are close
- 'education' entries were all replaced by their lower case versions
- 'XNA' were assumed to be 'X' gender, including all categories apart from M and F

## Dictionaries

### Introduction

Dictionaries are reference tables that identify the information contained in ID numbers. This can prove useful when changing these information to new categories. Instead of changing the values in each row, only the reference table is replaced.

### Creating dictionaries

There is no reason to establish an ID for information that is already cointained in each row. As a matter of maintaing good habits for data, reference tables will be created for 'family_status', 'education' and 'income_type'.

In [19]:
family_ref = df[['family_status_id', 'family_status']]
family_ref = family_ref.drop_duplicates().reset_index(drop=True)
family_ref

,family_status_id,family_status
0,0,married
1,1,civil partnership
2,2,widow / widower
3,3,divorced
4,4,unmarried


In [20]:
education_ref = df[['education_id', 'education']]
education_ref = education_ref.drop_duplicates().reset_index(drop=True)

In [21]:
df.drop(columns=['education', 'family_status'], inplace=True)

In [22]:
income_ref = df[['income_type']]
income_ref = income_ref.drop_duplicates().reset_index(drop=True)
income_ref = income_ref.reindex(
    reversed([4, 5, 0, 6, 2, 3, 1])
    ).reset_index(drop=True)
income_ref['income_type_id'] = income_ref.index
income_ref = income_ref[['income_type_id', 'income_type']]
df = df.merge(
    income_ref, on='income_type', how='left'
) 
df.drop(labels='income_type', axis=1, inplace=True)

### Conclusions
- Dictionaries were created for both family status and education, so it is easier to keep track of information
- Education could have a dictionary that scales with higher education, but this could create inconsistencies with new data
- Income type now has a dictionary that scales with safer types of income
- Only "id" columns were kept in the dataframe


## Data Categorization

### Introduction

Data categorization is necessary to improve data readability. It is especially necessary when dealing with quantitative variables such as income and age, otherwise this data can hardly be inserted in pivot tables.


### Creating categories
Categorization will be categorized according to the following topics:

- Do they have children?
- What is their age group?
- How much is their monthly income?
- What is their main purpose to apply for a loan?
- Are they employed for a long time?


In [23]:
def categorize(data_val, min_num, max_num, cat_num):
    if type(data_val) != type(1):
        return ValueError('Data value is not int!')
    step_val = (max_num - min_num)/cat_num
    groups = []
    for i in range(cat_num+1):
        groups.append(int(min_num+i*step_val))
    if data_val <= groups[0]:
        return str(groups[0]) + ' <'
    if data_val > groups[cat_num]:
        return str(groups[cat_num] + 1) + ' >'
    for i in range(len(groups)):
        if data_val <= groups[i + 1]:
            return str(groups[i] + 1) + '-' + str(groups[i+1])
    return ValueError('Error')

In [24]:
def has_children(children):
    if children > 0:
        return True
    else:
        return False


df['has_children'] = df['children'].apply(has_children)

min_num = 25
max_num = 65
cat_num = 4

df['age_group'] = df['dob_years'].apply(categorize, args=(min_num, max_num, cat_num))

min_num = df['total_income'].describe()['25%']
max_num = df['total_income'].describe()['75%']
cat_num = 5

df['total_income_group'] = df['total_income'].apply(categorize, args=(min_num, max_num, cat_num))
df['total_income_group'].value_counts()

17248 <        5382
31288 >        5381
25672-28479    3794
20056-22863    2022
17249-20055    2015
22864-25671    1753
28480-31287    1178
Name: total_income_group, dtype: int64

In [25]:
import nltk
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from collections import Counter

wordnet_lemma = WordNetLemmatizer()
english_stem = SnowballStemmer('english')

words = ' '.join(df['purpose'])
words = words.lower()
words = nltk.word_tokenize(words)
lemmas = [wordnet_lemma.lemmatize(word, pos='n') for word in words]
lemmas = [english_stem.stem(word) for word in lemmas]

def tokenize_purpose(phrase):    
    stem_list = [['residenti', 'hous'], ['estat', 'properti'], ['second-hand'], ['car'], ['educ', 'univers'], ['wed']]
    phrase = phrase.lower()
    tokens = nltk.word_tokenize(phrase)
    tokens = [wordnet_lemma.lemmatize(word, pos='n') for word in tokens]
    tokens = [english_stem.stem(word) for word in tokens]
    for word in tokens:
        if word in stem_list[0]:
            return 'housing'
        elif word in stem_list[1]:
            return 'property'
        elif word in stem_list[2]:
            return 'second-hand'
        elif word in stem_list[3]:
            return 'car'
        elif word in stem_list[4]:
            return 'education'
        elif word in stem_list[5]:
            return 'wedding'
        else:
            output = 'artifact'
    return output

df['purpose_stem'] = df['purpose'].apply(tokenize_purpose)
df['purpose_stem'].value_counts()

property       6413
housing        4427
education      4022
car            3347
wedding        2348
second-hand     968
Name: purpose_stem, dtype: int64

In [26]:
def veteran(days):
    if days > 365*5:
        return True
    else:
        return False

df['veteran'] = df['days_employed'].apply(veteran)
df['veteran'].value_counts()

False    14257
True      7268
Name: veteran, dtype: int64

### Conclusion

- Data has been categorized:
    - There is a column for children as boolean. 'has_children'
    - There are groups for age and total income. 'age_group' and 'total_income_group'
    - Purpose has been categorized according to stems. 'purpose_stem'
    - There is a boolean category for people in the same employment for too long.

## Rechecking Data


### Introduction

Duplicated rows can be a nuisance when analyzing data, as they may deviate results with information from the same customer. In this case, there isn't a unique identifier for each client, yet it seems unlikely that a customer has exactly the same entries. Therefore, they must be removed.

### Checking the data again

At this point all of the issues have been taken care of. Let us just check that everything looks good!

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21525 entries, 0 to 21524
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   children            21525 non-null  int64 
 1   days_employed       21525 non-null  int64 
 2   dob_years           21525 non-null  int64 
 3   education_id        21525 non-null  int64 
 4   family_status_id    21525 non-null  int64 
 5   gender              21525 non-null  object
 6   debt                21525 non-null  int64 
 7   total_income        21525 non-null  int64 
 8   purpose             21525 non-null  object
 9   income_type_id      21525 non-null  int64 
 10  has_children        21525 non-null  bool  
 11  age_group           21525 non-null  object
 12  total_income_group  21525 non-null  object
 13  purpose_stem        21525 non-null  object
 14  veteran             21525 non-null  bool  
dtypes: bool(2), int64(8), object(5)
memory usage: 2.3+ MB


In [28]:
duplicated_rows = df[df.duplicated()]
print(f"The percentage of duplicated rows is {duplicated_rows['debt'].count()/df.shape[0]:.2%}")

The percentage of duplicated rows is 0.33%


In [29]:
df = df.drop_duplicates().reset_index(drop=True)
df[df.duplicated()]

,children,days_employed,dob_years,education_id,family_status_id,gender,debt,total_income,purpose,income_type_id,has_children,age_group,total_income_group,purpose_stem,veteran


In [30]:
df.head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,debt,total_income,purpose,income_type_id,has_children,age_group,total_income_group,purpose_stem,veteran
0,1,8438,42,0,0,F,0,40620,purchase of the house,4,True,36-45,31288 >,housing,True
1,1,4025,36,1,0,F,0,17933,car purchase,4,True,36-45,17249-20055,car,True
2,0,5623,33,1,0,M,0,23342,purchase of the house,4,False,26-35,22864-25671,housing,True
3,3,4125,32,1,0,M,0,42821,supplementary education,4,True,26-35,31288 >,education,True
4,0,1630,53,1,1,F,0,25379,to have a wedding,0,False,46-55,22864-25671,wedding,False
5,0,926,27,0,1,M,0,40922,purchase of the house,2,False,26-35,31288 >,housing,False
6,0,2879,43,0,0,F,0,38484,housing transactions,2,False,36-45,31288 >,housing,True
7,0,153,50,1,0,M,0,21732,education,4,False,46-55,20056-22863,education,False
8,2,6930,35,0,1,F,0,15337,having a wedding,4,True,26-35,17248 <,wedding,True
9,0,2189,41,1,0,M,0,23108,purchase of the house for my family,4,False,36-45,22864-25671,housing,True


### Conclusions
- Duplicated data was within the customers with previously missing values, so it doesn't seem completely at random
    - It is still hard to identify the reason why the duplicates are there
    - The 'purpose' column does not support the hypothesis of coincidences
- They represented only 0.25% of the total data, so duplicates were deleted\
- The data looks sharp for analysis!

## Answering questions

1) Question:
- Is there a relation between having kids and repaying a loan on time?

In [31]:
def create_pivot_percentage(df, index, columns, values, margins):
    pivot_sum = df.pivot_table(index=index, columns=columns, values=values, aggfunc='sum', margins=margins)
    pivot_count = df.pivot_table(index=index, columns=columns, values=values, aggfunc='count', margins=margins)
    return pivot_sum/pivot_count

create_pivot_percentage(df, 'age_group', 'has_children', 'debt', True)

has_children,False,True,All
age_group,,,
25 <,0.106622,0.105572,0.106331
26-35,0.105523,0.106325,0.105962
36-45,0.080584,0.085503,0.083027
46-55,0.066581,0.074675,0.068135
56-65,0.056321,0.057432,0.056412
66 >,0.049475,0.057143,0.049858
All,0.075438,0.092082,0.081150


1) Answer:
- A comparison was made considering the percentage of defaulting based on whether they have kids with the total for each age group. The percentage of defaulting is pretty similar whether the person has kids or not, independent of age. The clear relation is that the younger the person, the more likely they will default. This is useful for credit score.

2) Question:
- Is there a relation between marital status and repaying a loan on time?

In [32]:
tempdf = df.merge(family_ref, on='family_status_id', how='left')

create_pivot_percentage(tempdf, 'age_group', 'family_status', 'debt', False)

family_status,civil partnership,divorced,married,unmarried,widow / widower
age_group,,,,,
25 <,0.095395,0.181818,0.095128,0.120253,0.000000
26-35,0.112132,0.095238,0.101088,0.117841,0.105263
36-45,0.103302,0.057851,0.077603,0.093264,0.064516
46-55,0.083144,0.070796,0.060200,0.089686,0.065574
56-65,0.061977,0.054054,0.054842,0.042683,0.066955
66 >,0.048544,0.102564,0.046243,0.026667,0.057554


2) Answer:
- For unmarried or widow/windower clients the same relation stands for age groups: younger clients are more likely to default on loans. 
- If they are divorced before they're 25 years-old, the percentage gets to it's highest of 18%. Elders get the second place.
- As for married or civil partnership, the percentage of people who have defaulted starts decreasing after they're 36 years-old.

3) Question:
- Is there a relation between income level and repaying a loan on time?

In [33]:
tempdf = df.merge(income_ref, on='income_type_id', how='left')

create_pivot_percentage(tempdf, 'total_income_group', 'income_type', 'debt', True)

income_type,business,civil servant,employee,paternity / maternity leave,retiree,student,unemployed,All
total_income_group,,,,,,,,
17248 <,0.082447,0.068182,0.095711,1.0,0.047455,0.0,1.0,0.079339
17249-20055,0.093995,0.077519,0.105359,NaN,0.049751,NaN,NaN,0.090323
20056-22863,0.066079,0.084615,0.100909,NaN,0.053254,NaN,NaN,0.084075
22864-25671,0.083333,0.026087,0.105263,NaN,0.066929,NaN,NaN,0.089561
25672-28479,0.073707,0.045977,0.101200,NaN,0.070755,NaN,NaN,0.085415
28480-31287,0.088235,0.089744,0.086601,NaN,0.074324,NaN,NaN,0.085739
31288 >,0.063922,0.048469,0.083694,NaN,0.060302,NaN,0.0,0.071734
All,0.074016,0.059025,0.095724,1.0,0.056412,0.0,0.5,0.081150


In [34]:
tempdf = df.merge(income_ref, on='income_type_id', how='left')

create_pivot_percentage(tempdf, 'income_type', 'veteran', 'debt', True).sort_values('All', ascending=False)

veteran,False,True,All
income_type,,,
paternity / maternity leave,NaN,1.000000,1.000000
unemployed,0.500000,NaN,0.500000
employee,0.109525,0.074977,0.095724
All,0.086846,0.070033,0.081150
business,0.080026,0.064614,0.074016
civil servant,0.063439,0.055944,0.059025
retiree,0.056412,NaN,0.056412
student,0.000000,NaN,0.000000


3) Answer:
- Besides very high total income values makes the client more trustworthy, there isn't much relation between defaulting on a loan and their income level.
- However, considering income type, 9.5% of the employees have defaulted on loans, followed by 7.5% of business owners, 5.9% of civil servants and 5.6% of retirees. Students and paternity / maternity leave do not have significant data for analysis.
- People with long periods in the same job tend to offer less risks, specially for employees and business owners.

4) Question:
- How do different loan purposes affect on-time repayment of the loan?

In [35]:
tempdf = df.merge(education_ref, on='education_id', how='left')
create_pivot_percentage(tempdf, 'purpose_stem', 'education', 'debt', True).sort_values('All', ascending=False)

education,bachelor's degree,graduate degree,primary education,secondary education,some college,All
purpose_stem,,,,,,
car,0.056533,NaN,0.139535,0.105616,0.111111,0.094554
education,0.058887,0.0,0.090909,0.103043,0.091603,0.092200
second-hand,0.091667,NaN,0.000000,0.091716,0.083333,0.090249
All,0.052952,0.0,0.109929,0.089902,0.091398,0.081150
wedding,0.045000,NaN,0.142857,0.092203,0.075000,0.080034
property,0.052465,0.0,0.092784,0.083222,0.085837,0.075829
housing,0.041894,0.0,0.148936,0.073718,0.095238,0.067271


4) Answer:
- Loans taken to buy cars or for education are more likely to get defaulted, with a percentage of 9.4%. Loans for properties and housing have the lowest rates, with 7.5% and 6.7%, respectively.
- People with lower levels of education have higher rates of defaulting, specially when the purpose are cars, housing or weddings.

## Conclusion

### Data Preprocess Tasks
- The data came with missing values in days_employed and total_income
    - days_employed had all of its values wrong in some way, so the code must be checked
- Marital status and education should have dictionaries, and income_type could use it as well
- Some entries are showing artifacts as 0 in age, this should be looked into 


### Analysis results
- Younger people tend to default more often, with rates varying from 6% for elders to 10% for the young.
- Kids didn't show a high impact on defaulting, whatever the age.
- The divorced young have a default rate of 18%, which is relatively very high.
- Unmarried clients show the same progression according to age, starting at 12% for the young and 2% for elders.
- Married and civil partners default rates rise to 11% and 10%, respectively, before slowly decreasing after the 25-35 year-old group.
- Divorced default rates reach their lowest rates at 36-45 years-old and 56-66 years-old, with 5.7% and 5.4%, respectively.
- Income ranges only significantly affect default rates for very high incomes values, when they lower.
- In order of risk of defaulting: employees, business owners, retirees and civil servants. 
- People with significant time in the same job have lower default rates, specially employees and business owners
- Loans to buy cars and education are more likely to get defaulted, while those for properties and housing are lowers, ranging from 9.4% to 6.7%
- Poeple with lower levels of education have higher default rates regarding, specially when the purpose are cars, wedding ceremonies and housing.
